In [12]:
import numpy as np
import scipy.signal
import scipy.io.wavfile as wav

def autocorrelation(x, max_lag):
    """Compute the autocorrelation of a signal."""
    result = np.correlate(x, x, mode='full')
    mid = len(result) // 2
    return result[mid:mid + max_lag + 1]

def levinson_durbin(r, order):
    """Solve the Yule-Walker equations using the Levinson-Durbin recursion."""
    a = np.zeros(order + 1)
    e = r[0]  # Initial error

    if e == 0:
        return a, e

    a[0] = 1  # LPC coefficient a0 is always 1
    for i in range(1, order + 1):
        acc = np.sum(a[:i] * r[i:0:-1])  # Compute reflection coefficient
        k = -acc / e  # Reflection coefficient
        a[1:i + 1] += k * a[i - 1::-1]  # Update coefficients
        e *= (1 - k ** 2)  # Update error

    return a, e

def lpc_analysis(signal, order):
    """Compute LPC coefficients for a given signal."""
    r = autocorrelation(signal, order)
    return levinson_durbin(r, order)

def synthesize_lpc(signal, lpc_coeffs):
    """Reconstruct signal using LPC coefficients (inverse filtering)."""
    return scipy.signal.lfilter([0] + -lpc_coeffs[1:], [1], signal)

from scipy.io.wavfile import read
if __name__ == "__main__":
    # Example: Load or generate a test signal
    fs = 8000  # Sample rate
    t = np.linspace(0, 1, fs, endpoint=False)
    # signal = np.sin(2 * np.pi * 200 * t) + 0.5 * np.random.randn(len(t))  # Synthetic speech-like signal
    a = read("samples/ahh.wav")
    signal = np.array(a[1],dtype=float)
    signal = signal[:,0]
    print(signal.shape)

    lpc_order = 10
    lpc_coeffs, error = lpc_analysis(signal, lpc_order)
    reconstructed_signal = synthesize_lpc(signal, lpc_coeffs)

    print("LPC Coefficients:", lpc_coeffs)


(93184,)
LPC Coefficients: [ 1.         -2.19031335  1.7570239  -0.53499565 -0.29110302  0.07221224
  0.40955299 -0.35390338 -0.07011612  0.43851699 -0.23468204]


In [13]:
import soundfile as sf
import sounddevice as sd


sd.play(reconstructed_signal, fs)